#Import lib and Generate_dataset class

In [73]:
import numpy as np
import matplotlib.pyplot as plt
# m: no of samples 
# n : no of features

In [74]:
# m: no of samples 
# n : no of features
class Generate_dataset:

  def getTrainSet(self, m, n):
    X = np.sort(np.random.uniform(0.0, 1.0, (m*n))).reshape(m, n) # sort will help to plot data properly 
    return X

  def getTestSet(self, X):
    m = X.shape[0]
    ep = np.random.normal(loc=0.0, scale =0.25, size =m) # eplsilon
    y = np.array([np.sin(2*np.pi*np.linalg.norm(X[i]))+ ep[i] for i in range(m)]).reshape(m,1)

    return y
    


#SVR class

In [78]:
class SVR:
  def rbf_kernal_fun(self, x,y):
    sigma= 1.2
    #K = np.exp(-(1 / (sigma**2)) * np.sum((x-y)**2))
    K = np.exp(-(1 / (sigma**2)) * np.linalg.norm(((x-y))**2))
    return K

  def construct_kernel_matrix(self, x1, x2, sigma=1):

    # applying kernel fun to each combinations of test set samples 
    K = np.zeros((x1.shape[0],x2.shape[0]))
    for i in range(x1.shape[0]):
      for j in range(x2.shape[0]):
        K[i,j] = self.rbf_kernal_fun(x1[i,:], x2[j,:])
    
    # K : m x m
    return K

    ''' 
    # X : m * n
    #||x-y||^2 = ||x||^2 + ||y||^2 - 2 * x * y^T : optimized way
    # https://stackoverflow.com/questions/47271662/what-is-the-fastest-way-to-compute-an-rbf-kernel-in-python

    X_norm = np.sum(X ** 2, axis = 1)
    K = np.exp(- (1/(sigma**2)) * (X_norm[:,None] + X_norm[None,:] - 2 * (X @ X.T)))'''

  def fit_direct(self, X,y, lamda):
    x1 = X[:, 0].reshape((X.shape[0],1))
    x2 = X[:, 1].reshape((X.shape[0],1))
    K = self.construct_kernel_matrix(x1,x2)
    I = np.identity(X.shape[0]+1) #np.eye(np.size( K , 1))

    #add col at end bias
    K = np.insert(K, -1, 1, axis=1) # for bias insert 1 extra col of 1s at lst col
    #I = np.insert(I, -1, 0, axis=1) # for bias insert 1 extra col of 0s at lst col
    print((K.T @ K).shape, K.T.shape, y.shape)
    theta = np.linalg.pinv( (K.T @ K) + (lamda * I )) @ (K.T @ y)

    return K, theta

  def predict_direct(self, xTest, xTrain, theta):

    K = self.construct_kernel_matrix(xTest,xTrain)
    K = np.insert(K, -1, 1, axis=1) # for bias insert 1 extra col of 1s at 0th col
    print(K.shape, theta.shape)
    Y_pred = K @ theta

    return Y_pred


  def find_rmse(self, y, y_pred):
    x = np.sqrt(np.mean((y_pred - y)**2))
    return x


  def pltGraph(self, X, y, y_pred):
    #fig = plt.figure(figsize=(9, 6))
    x1 = X[:, 0].reshape((X.shape[0],1))
    x2 = X[:, 1].reshape((X.shape[0],1))
    #print(x1.shape,x2.shape,y.shape,y_pred.shape )
    # Create 3D container
    ax = plt.axes(projection = '3d')
    ax.scatter3D(x1,x2,y, 'y')
    # Visualize 3D scatter plot
    ax.plot3D(x1, x2,y_pred.flatten() ,'g')

    # Give labels
    ax.set_xlabel('x1')
    ax.set_ylabel('X2')
    ax.set_zlabel('Y')

  def new_plot(self, X, y_test, y_pred):
    fig = plt.figure(figsize=(9, 6))
    x1 = X[:, 0].reshape((X.shape[0],1))
    x2 = X[:, 1].reshape((X.shape[0],1))
    # Create 3D container
    ax = plt.axes(projection = '3d')

    # Visualize 3D scatter plot
    ax.scatter3D(x1, x2,y_test)
    #ax.scatter3D(x1, x2,y_pred)

    # Give labels
    ax.set_xlabel('x1_train')
    ax.set_ylabel('X2_train')
    ax.set_zlabel('Y_train')

In [79]:
data = Generate_dataset()
xTrain = data.getTrainSet(400, 2)
yTrain = data.getTestSet(xTrain)

xTest = data.getTrainSet(200,2)
yTest = data.getTestSet(xTest)

my = SVR()
K, theta = my.fit_direct(xTrain, yTrain, 0.005)
y_pred = my.predict_direct(xTest, xTrain, theta)
x=my.find_rmse(yTest,y_pred)
x

(401, 401) (401, 400) (400, 1)
(200, 401) (401, 1)


6.403027838308689

#consider till above